In [2]:
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import Nadam
from keras.preprocessing.image import ImageDataGenerator
import h5py
import numpy as np
import cv2

DATA_FILE = 'data.h5df'

In [3]:
with h5py.File(DATA_FILE, 'r') as f:
    
    #TODO: gather more training data on neighborhood
    images_training = list(f['images_training'])
    labels_training = np.array(f['label_training'])
    
    images_testing = list(f['images_testing'])
    labels_testing = np.array(f['label_testing'])

In [4]:
"""
Let's reshape the image so it is faster to train.
"""

def resize_list_imgs(dset, new_size=(128, 128)):
    dset = list(dset)
    for i in range(len(dset)):
        dset[i] = cv2.resize(dset[i], new_size)[..., np.newaxis]
    return np.array(dset)
    
images_training = resize_list_imgs(images_training)
images_testing = resize_list_imgs(images_testing)

In [5]:
image_input_shape = images_training[0].shape

activation = 'relu'

pic_input = Input(shape=image_input_shape)

# TODO: add more conv net layers, multiple color frame input layer

img_stack = Conv2D(32, (3, 3), name="convolution0", padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name="convolution1", padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.2)(img_stack)

dense_stack = Dense(64, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.2)(dense_stack)
dense_stack = Dense(10, activation=activation, name='dense2')(dense_stack)
dense_stack = Dropout(0.2)(dense_stack)
dense_stack = Dense(3, name='output')(dense_stack)

adam = Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss='mse')

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 131072)            0         
_________________________________________________________________
dense0 (Dense)               (None, 64)                8388672   
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
__________

In [10]:
# TODO: custom generator looking at ROI and flipping images
generator = ImageDataGenerator()
generator.fit(images_training)
batch_size = 32

model.fit_generator(generator.flow(images_training, labels_training, batch_size=batch_size),
                    steps_per_epoch=len(images_training)//batch_size, epochs=200, verbose=1)

Epoch 1/200
14/14 [==============================] - 1s 88ms/step - loss: 0.0950
Epoch 2/200
14/14 [==============================] - 1s 90ms/step - loss: 0.0935
Epoch 3/200
14/14 [==============================] - 1s 85ms/step - loss: 0.0896
Epoch 4/200
14/14 [==============================] - 1s 84ms/step - loss: 0.0984
Epoch 5/200
14/14 [==============================] - 1s 87ms/step - loss: 0.0994
Epoch 6/200
14/14 [==============================] - 1s 87ms/step - loss: 0.0894
Epoch 7/200
14/14 [==============================] - 1s 88ms/step - loss: 0.0947
Epoch 8/200
14/14 [==============================] - 1s 83ms/step - loss: 0.0872
Epoch 9/200
14/14 [==============================] - 1s 87ms/step - loss: 0.1060
Epoch 10/200
14/14 [==============================] - 1s 87ms/step - loss: 0.0940
Epoch 11/200
14/14 [==============================] - 1s 102ms/step - loss: 0.0970
Epoch 12/200
14/14 [==============================] - 1s 90ms/step - loss: 0.0931
Epoch 13/200
14/14 [====

14/14 [==============================] - 6s 400ms/step - loss: 0.1007
Epoch 101/200
14/14 [==============================] - 5s 388ms/step - loss: 0.0873
Epoch 102/200
14/14 [==============================] - 6s 403ms/step - loss: 0.0951
Epoch 103/200
14/14 [==============================] - 6s 414ms/step - loss: 0.0913
Epoch 104/200
14/14 [==============================] - 6s 408ms/step - loss: 0.0978
Epoch 105/200
14/14 [==============================] - 5s 392ms/step - loss: 0.0950
Epoch 106/200
14/14 [==============================] - 6s 393ms/step - loss: 0.0929
Epoch 107/200
14/14 [==============================] - 5s 385ms/step - loss: 0.0933
Epoch 108/200
14/14 [==============================] - 6s 394ms/step - loss: 0.0918
Epoch 109/200
14/14 [==============================] - 5s 382ms/step - loss: 0.1025
Epoch 110/200
14/14 [==============================] - 5s 391ms/step - loss: 0.0895
Epoch 111/200
14/14 [==============================] - 6s 395ms/step - loss: 0.0981
Epoch 

14/14 [==============================] - 5s 370ms/step - loss: 0.0955
Epoch 198/200
14/14 [==============================] - 6s 403ms/step - loss: 0.0923
Epoch 199/200
14/14 [==============================] - 5s 362ms/step - loss: 0.0968
Epoch 200/200
14/14 [==============================] - 5s 364ms/step - loss: 0.0861


In [11]:
save_model(model, 'model.h5df')